<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigDataII_01_bigdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

#　以下好きなフォルダを作って指定する
%cd "/content/drive/MyDrive/Lecture_BigData"

# 大規模データの処理

In [ ]:
import numpy as np
import pandas as pd

# 以下データを置くフォルダを好きに作って指定する
DIR_NAME = 'dataset_temp/'

COL_NAME = [str(i) for i in range(0,5)]

testdata = np.random.rand(100, 5)
df = pd.DataFrame(testdata, columns=COL_NAME)
filename = DIR_NAME + 'testcsv.csv'
df.to_csv(filename, index=False)

df = pd.read_csv(filename, encoding='utf-8')

df

In [ ]:
# ファイルまたはフォルダのサイズを表示
!du -h dataset_temp/testcsv.csv

## 0. Chunkに分けてファイルを読み込む

In [ ]:
i = 0
# chunkに分けて処理
for df in pd.read_csv(filename, encoding='shift-jis', chunksize=25):
    print(df.shape)
    print(df)
    
    # chunkごとに分けて処理
    df['5'] = 'chunk ' + str(i)
    df.to_csv(DIR_NAME+'testcsv_processed.csv', mode='a', index=False, header=(i == 0))
    i += 1

# データの準備

In [ ]:
import numpy as np 
import pandas as pd

DIR_NAME = 'dataset_temp/'

#乱数作成
for i in range(100):
    testdata = np.random.rand(100,100)      # 0〜1の乱数で 100x100 の行列を生成
    df = pd.DataFrame(testdata)             #dataframeに変換
    filename = DIR_NAME + 'dammydata' + str(i).zfill(5) + '.csv'
    df.to_csv(filename , index=False)       #csvに保存

In [ ]:
! du -h

# 1.   For loopで読み込む　pandas.read_csv forloop

In [ ]:
#pandas.read_csv map
import time
import glob
import numpy as np 
import pandas as pd

#map(pd.concat)
def read_csv_map(fileslist):
    df = pd.concat(map(pdreadcsv, fileslist))
    return df

#csv1個読み込み(map関数用)
def pdreadcsv(csv_path):
    return pd.read_csv(csv_path, encoding='utf-8')

if __name__ == "__main__":
    
    allfiles = sorted(glob.glob(DIR_NAME+'*.csv', recursive=True))
    
    start = time.time()

    df = read_csv_map(allfiles)

    process_time = time.time() - start
    print('csv読み込み時間：{:.3f}s'.format(process_time))

csv読み込み時間：1.075s


# 2．Mapを使って読み込む

In [ ]:
# map のサンプル
# 参照：https://qiita.com/conf8o/items/0cb02bc504b51af09099

data = [1,2,3,4,5]

def double(x):
  return x*x

print(double(data[2]))

m = map(double, data)
l = list(m)
print(l)

In [ ]:
#pandas.read_csv map
import time
import glob
import numpy as np 
import pandas as pd

#map(pd.concat)
def read_csv_map(fileslist):
    #---code here---#
    m = map(pdreadcsv, fileslist)
    df = pd.concat(m)
    return df

#csv1個読み込み(map関数用)
def pdreadcsv(csv_path):
    return pd.read_csv(csv_path, encoding='utf-8')

if __name__ == "__main__":
    
    allfiles = sorted(glob.glob(DIR_NAME+'*.csv', recursive=True))
    
    start = time.time()

    df = read_csv_map(allfiles)

    process_time = time.time() - start
    print('csv読み込み時間：{:.3f}s'.format(process_time))

# マルチプロセスの例

メモ

process は、複数の関数を複数プロセスで並列して実行します。実行中の関数は全てメモリ上に展開されます。

pool は、一つの関数に複数の処理を行わせる際に、その処理を複数プロセスに割り当てて並列して実行します。pool 側でタスクの分割や結果の統合といったことを暗黙的に制御し、実行中の処理のみがメモリ上に展開されます。

In [ ]:
import time 

def sum_cube(num):
    s = 0
    for i in range(num):
        s += i * i * i
    return s

def return_list_sum_cube(numbers):
    start = time.time()
    result = []
    for i in numbers:
        result.append(sum_cube(i))

    end = time.time() - start
    print(f'No Multiprocessing:  {end} seocnds')

    return result

if __name__ == '__main__':

    numbers = range(10)
    results = return_list_sum_cube(numbers)
    print(results)

In [ ]:
import time 
import multiprocessing 

def sum_cube(num):
    s = 0
    for i in range(num):
        s += i * i * i
    return s

def return_list_sum_cube_with_multiprocessing(numbers):
    start = time.time()
    
    #---code here---#
    p = multiprocessing.Pool()
    result = p.map(sum_cube, numbers)

    p.close()
    p.join()

    end = time.time() - start
    print(f'Multiprocessing:  {end} seocnds')

    return result

if __name__ == '__main__':

    numbers = range(10)
    result = return_list_sum_cube_with_multiprocessing(numbers)
    print(result)

# 3. マルチプロセスでpandas.read_csvをmapで実行してpandas.concatで結合

In [ ]:
#pandas.read_csv map multiprocessing
import time
import glob
import numpy as np 
import pandas as pd
import os
from multiprocessing import Pool

#map_multiprocessing(pd.concat)
def read_csv_map_multi(fileslist):
    p = Pool(os.cpu_count())
    df = pd.concat(p.map(pdreadcsv, fileslist))
    p.close()
    return df


#csv1個読み込み(map関数用)
def pdreadcsv(csv_path):
    return pd.read_csv(csv_path, encoding='utf-8')


if __name__ == "__main__":
  
    allfiles = sorted(glob.glob(DIR_NAME+'dammy*.csv', recursive=True))

    start = time.time()

    df = read_csv_map_multi(allfiles)

    process_time = time.time() - start
    print('csv読み込み時間：{:.3f}s'.format(process_time))

csv読み込み時間：0.853s


# 4. マルチプロセスでpandas.read_csvをmapで実行してnumpy.vstackで結合

In [ ]:
#readcsv_pandas_np.vstack map multi
import time
import glob
import numpy as np 
import pandas as pd
import os
from multiprocessing import Pool

#map_multiprocessing(np.vstack)
def read_csv_map_multi_npvstack(fileslist):
    p = Pool(os.cpu_count())
    comb_np_array = np.vstack(p.map(pdreadcsv_np_array, fileslist))
    df = pd.DataFrame(comb_np_array)    
    p.close()
    
    return df

def pdreadcsv_np_array(csv_path):
    df = pd.read_csv(csv_path, encoding='utf-8')
    np_array = df.values
    return np_array 
  
if __name__ == "__main__":
    
    allfiles = sorted(glob.glob(DIR_NAME+'dammy*.csv', recursive=True))  
    
    start = time.time()

    df = read_csv_map_multi_npvstack(allfiles)

    process_time = time.time() - start
    print('csv読み込み時間：{:.3f}s'.format(process_time))

大量のcsvファイルを高速に読み込む方法を検討しました。

今回紹介した方法が必ずしもベストではなく、csvファイルのサイズとファイル数によって読み込み速度は異なってきます。

単純なfor文は想定通り遅く、mapやリスト内包表記を使用することで速度アップができました。

また並列処理することでより高速に読み込むことができました。ただし並列処理はＣＰＵ使用率が大幅ＵＰするデメリットもあるので注意が必要です。